<a href="https://colab.research.google.com/github/LucasRios/AIPhytonExamples/blob/main/m%C3%A9tricas_e_avalia%C3%A7%C3%A3o_de_aprendizado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

In [ ]:
# Download do dataset
!wget --no-check-certificate https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

#extração
local_zip = '/content/kagglecatsanddogs_5340.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

#limpeza de imagens corrompidas
num_skipped = 0
for folder_name in ("PetImages/Cat", "PetImages/Dog"):
  folder_path = os.path.join("/content", folder_name)
  for fname in os.listdir(folder_path):
    fpath = os.path.join(folder_path, fname)
    try:
        fobj = open(fpath, "rb")
        is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
    finally:
        fobj.close()
    if not is_jfif:
        num_skipped += 1
        os.remove(fpath)
print("Imagens corrompidas removidas: %d" % num_skipped)


--2025-12-26 18:15:33--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.222.202.7, 2600:1408:c400:f95::317f, 2600:1408:c400:f86::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.222.202.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip.1’

kagglecatsanddogs_5 100%[===================>] 786.67M   127MB/s    in 8.7s    

2025-12-26 18:15:42 (90.6 MB/s) - ‘kagglecatsanddogs_5340.zip.1’ saved [824887076/824887076]



In [ ]:
#Preparação dos Dados (Data Generators)
base_dir = 'PetImages'
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # 20% para teste

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224,224),
    batch_size=30,
    class_mode='binary',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)


In [ ]:
#Construindo o Modelo com Transfer Learning
vgg_base = tf.keras.applications.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

vgg_base.trainable = False

model = models.Sequential([
    vgg_base, layers.Flatten(), layers.Dense(256, activation='relu'), layers.Dropout(0.5), layers.Dense(1, activation='sigmoid')

])

model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

model.summary()

In [ ]:
#Treinamento
history = model.fit(train_generator, epochs=2, validation_data=validation_generator)

In [ ]:
## 1. Gerar previsões
Y_pred = model.predict(validation_generator)
y_pred = (Y_pred > 0.5).astype(int).ravel()
y_true = validation_generator.classes
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))

# 2. Obter a Matriz de Confusão
# No scikit-learn: TN, FP, FN, TP
tn,fp,fn,tp = confusion_matrix(y_true, y_pred).ravel()

# 3. Funções das Métricas (Implementação Manual)
def mostrar_metricas(vp,vn,fp,fn):
    print('Taxa de Verdadeiro Positivo: ', vp)
    print('Taxa de Verdadeiro Negativo: ', vn)
    print('Taxa de Falso Positivo: ', fp)
    print('Taxa de Falso Negativo: ', fn)

    acuracia = (vp + vn) / (vp + vn + fp + fn)
    print('Acurácia: ', acuracia)

    precisao = vp / (vp + fp)
    print('Precisão: ', precisao)

    sensibilidade = vp / (vp + fn)
    print('Sensibilidade: ', sensibilidade)

    especificidade = vn / (vn + fp)
    print('Especificidade: ', especificidade)

    f_score = 2 * (precisao * sensibilidade) / (precisao + sensibilidade)
    print('F-Score: ', f_score)

# Exibir resultados reais do modelo
mostrar_metricas(tp, tn, fp, fn)